In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

2022-04-18 15:13:00.032010: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-18 15:13:00.032148: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class modelNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [16]:
EPOCHS = 15
INIT_LR = 1e-3
BS = 64

In [4]:
print("[INFO] loading images...")
data = []
labels = []

[INFO] loading images...


In [5]:
imagePaths = sorted(list(paths.list_images("augData")))
random.seed(42)
random.shuffle(imagePaths)

In [6]:
for imagePath in imagePaths:    
    label = imagePath.split(os.path.sep)[-2]
    
    if label == "triple_rider":
        label = 1
        labels.append(label)
    else:
        label = 0
        labels.append(label)
    
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    

In [7]:
unique, counts = np.unique(np.array(labels), return_counts=True)
dict(zip(unique, counts))

{0: 5015, 1: 4652}

In [8]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [9]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [136]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	fill_mode="nearest")

In [17]:
print("[INFO] compiling model...")
model = modelNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [18]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, batch_size=BS,
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)
# save the model to disk
print("[INFO] serializing network...")
model.save("model_1.h5", save_format="h5")

[INFO] training network...
Epoch 1/15
113/113 [==============================] - 27s 225ms/step - loss: 0.5284 - accuracy: 0.7573 - val_loss: 0.4883 - val_accuracy: 0.7865
Epoch 2/15
113/113 [==============================] - 24s 216ms/step - loss: 0.4569 - accuracy: 0.7999 - val_loss: 0.4856 - val_accuracy: 0.7592
Epoch 3/15
113/113 [==============================] - 24s 213ms/step - loss: 0.3874 - accuracy: 0.8348 - val_loss: 0.3837 - val_accuracy: 0.8407
Epoch 4/15
113/113 [==============================] - 24s 213ms/step - loss: 0.3368 - accuracy: 0.8575 - val_loss: 0.3336 - val_accuracy: 0.8614
Epoch 5/15
113/113 [==============================] - 24s 213ms/step - loss: 0.2731 - accuracy: 0.8820 - val_loss: 0.3545 - val_accuracy: 0.8271
Epoch 6/15
113/113 [==============================] - 25s 220ms/step - loss: 0.2474 - accuracy: 0.9022 - val_loss: 0.3707 - val_accuracy: 0.8573
Epoch 7/15
113/113 [==============================] - 25s 220ms/step - loss: 0.1890 - accuracy: 0.9261 

In [37]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Double and triple rider")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot_2.png")

In [20]:
predictions = model.predict(x=testX, batch_size=32)

print(predictions.shape)

(2417, 2)


In [25]:
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1),target_names=["triple_rider","double_rider"]))

              precision    recall  f1-score   support

triple_rider       0.90      0.91      0.91      1269
double_rider       0.90      0.89      0.90      1148

    accuracy                           0.90      2417
   macro avg       0.90      0.90      0.90      2417
weighted avg       0.90      0.90      0.90      2417



In [28]:
#importing the original dataset
data = []
labels = []
imagePaths = sorted(list(paths.list_images("Dataset")))

for imagePath in imagePaths:
    
    label = imagePath.split(os.path.sep)[-2]
    if label == "triple_person":
        label = 1
    else:
        label = 0
    labels.append(label)
    
    # for images
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)


In [31]:
data = np.array(data)
labels = np.array(labels)

labels = to_categorical(labels, num_classes=2)

In [34]:
predictions = model.predict(x=data, batch_size=64)


In [38]:
print(classification_report(labels.argmax(axis=1), predictions.argmax(axis=1),
                        target_names=["double_rider","triple_rider"]))

              precision    recall  f1-score   support

double_rider       0.94      0.82      0.87       948
triple_rider       0.06      0.17      0.09        63

    accuracy                           0.78      1011
   macro avg       0.50      0.50      0.48      1011
weighted avg       0.88      0.78      0.82      1011

